In [5]:
import os
os.chdir('..')

# Import necessary modules

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read dataset

In [77]:
from sklearn.preprocessing import MinMaxScaler

In [78]:
species_map = {'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [79]:
df = pd.read_csv('example/data.csv')

In [80]:
df_X = df.drop(columns='species')
df_y = df['species'].map(species_map)

In [81]:
df_X

,sepal-length,sepal-width,petal-length,petal-width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [82]:
df_y

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: species, Length: 150, dtype: int64

In [83]:
x = df_X.values 
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_X = pd.DataFrame(x_scaled, columns=df_X.columns)

In [84]:
df_X

,sepal-length,sepal-width,petal-length,petal-width
0,0.222222,0.625000,0.067797,0.041667
1,0.166667,0.416667,0.067797,0.041667
2,0.111111,0.500000,0.050847,0.041667
3,0.083333,0.458333,0.084746,0.041667
4,0.194444,0.666667,0.067797,0.041667
...,...,...,...,...
145,0.666667,0.416667,0.711864,0.916667
146,0.555556,0.208333,0.677966,0.750000
147,0.611111,0.416667,0.711864,0.791667
148,0.527778,0.583333,0.745763,0.916667


# Create linguistic variables

In [48]:
from doggos.knowledge import LinguisticVariable, Domain

In [145]:
# features linguistic variables
sepal_length = LinguisticVariable('sepal-length', Domain(0, 1.001, 0.001))
sepal_width = LinguisticVariable('sepal-width', Domain(0, 1.001, 0.001))
petal_length = LinguisticVariable('petal-length', Domain(0, 1.001, 0.001))
petal_width = LinguisticVariable('petal-width', Domain(0, 1.001, 0.001))

In [146]:
# decision linguistic variable
species = LinguisticVariable('species', Domain(0, 1, 0.001))

# Define clauses

In [147]:
from doggos.knowledge import Clause
from doggos.fuzzy_sets import IntervalType2FuzzySet
from doggos.utils.membership_functions import triangular, trapezoidal, gaussian

In [148]:
clauses = [
    # clauses for petal length
    Clause(petal_length, 'small', IntervalType2FuzzySet(triangular(0, 0.25, 0.5, 0.8), triangular(0, 0.25, 0.5))),
    Clause(petal_length, 'medium', IntervalType2FuzzySet(triangular(0.25, 0.5, 0.75, 0.8), triangular(0.25, 0.5, 0.75))),
    Clause(petal_length, 'big', IntervalType2FuzzySet(triangular(0.5, 0.75, 1, 0.8), triangular(0.5, 0.75, 1, 1))),
    # clauses for petal width
    Clause(petal_width, 'small', IntervalType2FuzzySet(gaussian(0.25, 0.1), gaussian(0.25, 0.15))),
    Clause(petal_width, 'medium', IntervalType2FuzzySet(gaussian(0.5, 0.1), gaussian(0.5, 0.15))),
    Clause(petal_width, 'big', IntervalType2FuzzySet(gaussian(0.75, 0.1), gaussian(0.75, 0.15))),
    # claueses for sepal length
    Clause(sepal_length, 'small', IntervalType2FuzzySet(triangular(0, 0.25, 0.5, 0.8), triangular(0, 0.25, 0.5))),
    Clause(sepal_length, 'medium', IntervalType2FuzzySet(triangular(0.25, 0.5, 0.75, 0.8), triangular(0.25, 0.5, 0.75))),
    Clause(sepal_length, 'big', IntervalType2FuzzySet(triangular(0.6, 0.9, 1, 0.8), triangular(0.6, 0.9, 1, 1))),
    # clauses for sepal width
    Clause(sepal_width, 'small', IntervalType2FuzzySet(gaussian(0.25, 0.1), gaussian(0.25, 0.15))),
    Clause(sepal_width, 'medium', IntervalType2FuzzySet(gaussian(0.5, 0.1), gaussian(0.5, 0.15))),
    Clause(sepal_width, 'big', IntervalType2FuzzySet(gaussian(0.75, 0.1), gaussian(0.75, 0.15)))
]

In [149]:
for clause in clauses:
    print(clause)

Clause petal-length is small
Clause petal-length is medium
Clause petal-length is big
Clause petal-width is small
Clause petal-width is medium
Clause petal-width is big
Clause sepal-length is small
Clause sepal-length is medium
Clause sepal-length is big
Clause sepal-width is small
Clause sepal-width is medium
Clause sepal-width is big


In [150]:
# decision clause

In [151]:
decision_clauses = [
    Clause(species, 'zero', IntervalType2FuzzySet(gaussian(0.25, 0.1), gaussian(0.25, 0.15))),
    Clause(species, 'one', IntervalType2FuzzySet(gaussian(0.5, 0.1), gaussian(0.5, 0.15))),
    Clause(species, 'two', IntervalType2FuzzySet(gaussian(0.75, 0.1), gaussian(0.75, 0.15)))
]

# Define consequents

In [152]:
from doggos.knowledge.consequents import MamdaniConsequent

In [153]:
species_zero = MamdaniConsequent(decision_clauses[0])
species_one = MamdaniConsequent(decision_clauses[1])
species_two = MamdaniConsequent(decision_clauses[2])

# Define terms

In [154]:
from doggos.knowledge import Term
from doggos.algebras import LukasiewiczAlgebra

In [155]:
terms = list()
algebra = LukasiewiczAlgebra()
for clause in clauses:
    terms.append(Term(algebra, clause))

# Define antecedents

In [156]:
# define random antecedents
antecedent1 = terms[0] & terms[3] | terms[5]
antecedent2 = (terms[1] & terms[4]) | (terms[3] & terms[8])
antecedent3 = terms[2] & (terms[0] | terms[7] & (terms[3] | terms[6]) | terms[8])

# Create rules

In [157]:
from doggos.knowledge import Rule

In [158]:
rules = [
    Rule(antecedent1, species_zero),
    Rule(antecedent2, species_one),
    Rule(antecedent3, species_two)
]

# Fuzzify dataset

In [159]:
from doggos.knowledge import fuzzify

In [160]:
df_X_fuzzified = fuzzify(df_X, clauses)

# Create inference system

In [163]:
from doggos.inference import MamdaniInferenceSystem
from doggos.inference.defuzzification_algorithms import karnik_mendel

In [164]:
inference_system = MamdaniInferenceSystem(rules)

In [165]:
inference_system.infer(karnik_mendel, df_X_fuzzified)

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed